## ODOC Public Inmate Data


This notebook is intended as a start for research of the ODOC data.  To use this notebook, follow the setup instructiion found [here](https://github.com/codefortulsa/odoc-parse). Then download the data published [here](http://doc.publishpath.com/odoc-public-inmate-data).  Unzip the file and place the files in a subdirectory called 'data'.

The set of files includes a ReadMe.txt which describes the files and their fixed formats. The sections of this notebook show description of each file and how to import it into pandas dataframes.  NOTE: the widths variables differ slighty from the description to handle some difference in the data.

In [1]:
import pandas as pd

In [22]:
file = 'data/Vendor_Profile_Sample_Text.DAT'

# uncomment this line to use the full dataset
file = 'data/Vendor_Profile_Extract_Text.DAT'

names = [
    "DOC_NUM"       
    ,"LAST_NAME"     
    ,"FIRST_NAME"    
    ,"MIDDLE_NAME"   
    ,"SUFFIX"        
    ,"LAST_MOVE_DATE"
    ,"FACILITY"      
    ,"BIRTH_DATE"    
    ,"SEX"           
    ,"RACE"          
    ,"HAIR"          
    ,"HEIGHT_FT"     
    ,"HEIGHT_IN"     
    ,"WEIGHT"        
    ,"EYE"           
    ,"STATUS"
]
widths = [
    11,
    30,
    30,
    30,
    5,
    9,
    40,
    9,
    1,
    40,
    40,
    2,
    2,
    4,
    40,10
]

profile_df = pd.read_fwf(file, 
    header=None,
    widths=widths,
    names=names)

profile_df

from datetime import datetime

file = 'data/Vendor_sentence_Sample_Text.dat'

# uncomment this line to use the full dataset
file = 'data/Vendor_sentence_Extract_Text.dat'


names =[
    "DOC_NUM",                   
    "STATUTE_CODE",              
    "SENTENCING_COUNTY",         
    "JS_DATE",                   
    "CRF_NUMBER",                
    "INCARCERATED_TERM_IN_YEARS",
    "PROBATION_TERM_IN_YEARS"
]

widths = [
    10,
    40,
    40,
    10,
    40,
    13,
    13
]

sentence_df = pd.read_fwf(file, 
    header=None,
    widths=widths,
    names=names)


combined_data = pd.merge(sentence_df, profile_df, on='DOC_NUM', how='left')

combined_data.dropna(subset=['LAST_NAME'], inplace=True)  
combined_data

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS
4527,8359,163-2-401,OKLAHOMA COUNTY COURT,10-JUN-87,87-615,5.00,NaN,MESLES,ARDELL,NaN,...,INACTIVE,06-APR-87,M,NaN,NaN,NaN,NaN,NaN,NaN,Inactive
36683,10337,021-1720,MCCURTAIN COUNTY COURT,04-OCT-79,79-167,1.00,2.0,SAWYER,FRANK,N,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
36684,10337,121-1435,MCCURTAIN COUNTY COURT,05-OCT-79,77-134,2.00,NaN,SAWYER,FRANK,N,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
36685,10337,221-1116,POTTAWATOMIE COUNTY COURT,05-SEP-79,79-220,NaN,2.0,SAWYER,FRANK,N,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
36686,10337,421-1431,POTTAWATOMIE COUNTY COURT,03-JAN-92,91-197,10.00,NaN,SAWYER,FRANK,N,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335211,99389,021-843.1,MISSOURI JURISDICTION,04-DEC-87,86-1307,5.00,NaN,LYONS,BEN,NaN,...,INACTIVE,19-NOV-60,M,BLACK,BLACK,5.0,8.0,165.0,BROWN,Inactive
1335212,99389,021-843.1,MISSOURI JURISDICTION,04-DEC-87,86-1307,5.00,NaN,LYONS,BEN,NaN,...,INACTIVE,19-NOV-60,M,BLACK,BLACK,5.0,8.0,165.0,BROWN,Inactive
1335213,99628,121-798,UNKNOWN COUNTY COURT,08-JUN-82,9-897,0.27,NaN,CADY,EDDIE,R,...,INACTIVE,02-JUN-58,M,WHITE,BROWN,6.0,1.0,209.0,N/R,Inactive
1335214,99628,121-798,UNKNOWN COUNTY COURT,08-JUN-82,9-897,7777.00,NaN,CADY,EDDIE,R,...,INACTIVE,02-JUN-58,M,WHITE,BROWN,6.0,1.0,209.0,N/R,Inactive


In [27]:
import oscn
import re


counties = oscn.counties

def make_date (dataDate):
    try:
        date = dataDate.split('-')
        year = date[2]
        if int(year) > 50:
            date[2]="19"+year
        else:
            date[2]="20"+year

        fixedDate = "-".join(date)
        dt = datetime.strptime(fixedDate, "%d-%b-%Y")
        return dt
    except:
#         print(dataDate)
        return None

    
# combined_data['JS_DATE'] = combined_data['JS_DATE'].apply(make_date)
# combined_data.sort_values(combined_data.columns[3], inplace=True)


def parse_county(dataCounty):
    if "JURISDICTION" in dataCounty:
#         print(dataCounty)
        return None
    county = dataCounty.split(" ")[0].lower()
    if county in counties:
        return county
#     print(dataCounty)
    return None

def parse_year(crf_number):
    split_number = re.split("-| ", crf_number)
    for i in range(2):
        try:
            maybe_year = int(split_number[i])
            if (maybe_year < 2021 and maybe_year > 1900):
                return split_number[i]
            if (maybe_year < 2021 and maybe_year > 1900):
                return split_number[i]
            if (maybe_year < 100 and maybe_year > 20):
                return "19" + split_number[i]
            if (maybe_year < 10 and maybe_year >= 0):
                return "200" + split_number[i]
            if (maybe_year <= 20 and maybe_year >= 10):
                return "20" + split_number[i]
        except:
            continue
#     print(crf_number)
    return None
    
def parse_number(crf_number):
    split_number = crf_number.split("-")
    return split_number[len(split_number)-1]


combined_data['oscn_county'] = combined_data['SENTENCING_COUNTY'].apply(parse_county)
combined_data.dropna(subset=['oscn_county'], inplace=True)  
combined_data['oscn_year'] = combined_data['CRF_NUMBER'].apply(parse_year)
combined_data['oscn_number'] = combined_data['CRF_NUMBER'].apply(parse_number)

combined_data

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS,oscn_county,oscn_year,oscn_number
4527,8359,163-2-401,OKLAHOMA COUNTY COURT,10-JUN-87,87-615,5.0,NaN,MESLES,ARDELL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Inactive,oklahoma,1987,615
36683,10337,021-1720,MCCURTAIN COUNTY COURT,04-OCT-79,79-167,1.0,2.0,SAWYER,FRANK,N,...,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive,mccurtain,1979,167
36684,10337,121-1435,MCCURTAIN COUNTY COURT,05-OCT-79,77-134,2.0,NaN,SAWYER,FRANK,N,...,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive,mccurtain,1977,134
36685,10337,221-1116,POTTAWATOMIE COUNTY COURT,05-SEP-79,79-220,NaN,2.0,SAWYER,FRANK,N,...,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1979,220
36686,10337,421-1431,POTTAWATOMIE COUNTY COURT,03-JAN-92,91-197,10.0,NaN,SAWYER,FRANK,N,...,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1991,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334839,85741,821-1713,LINCOLN COUNTY COURT,24-FEB-20,19-30A,NaN,7.0,BARNETT,NOVA,L,...,NATIVE AMERICAN,BLACK,5.0,8.0,240.0,BROWN,Inactive,lincoln,2019,30A
1334840,85741,963-2-401,LINCOLN COUNTY COURT,02-MAR-20,18-156,NaN,10.0,BARNETT,NOVA,L,...,NATIVE AMERICAN,BLACK,5.0,8.0,240.0,BROWN,Inactive,lincoln,2018,156
1335188,99018,421-798,WASHINGTON COUNTY COURT,16-MAY-84,84-6869,20.0,NaN,HESS,BILLY,JOE,...,WHITE,BROWN,5.0,8.0,145.0,BLUE,Inactive,washington,1984,6869
1335189,99018,421-645,WASHINGTON COUNTY COURT,16-MAY-84,84-1686,10.0,NaN,HESS,BILLY,JOE,...,WHITE,BROWN,5.0,8.0,145.0,BLUE,Inactive,washington,1984,1686


In [34]:
def get_oscn_data(oscn_county, oscn_year, oscn_number):
    try:
        case = oscn.request.Case(f'{oscn_county}-CF-{oscn_year}-{oscn_number}')
        return case.valid
    except:
        print(f'{oscn_county}-CF-{oscn_year}-{oscn_number}')
        return None

combined_data['oscn_valid'] = combined_data.apply(lambda x: get_oscn_data(x['oscn_county'], x['oscn_year'], x['oscn_number']), axis=1)

combined_data

creek-1999-D
creek-1999-18B
mcclain-2002-0172 CT. 1
mcclain-2002-0172 CT. 2
custer-2003-105B
oklahoma-2003-6577R
oklahoma-2003-6577R
payne-2010-486R
payne-2010-486R
payne-2010-486R
cherokee-2002-531R2
muskogee-2014-486R
muskogee-2014-486R
muskogee-2014-576R
muskogee-2014-576R
delaware-2003-142B
delaware-2003-142B
delaware-2003-142B
garfield-2003-551RR
garfield-2003-589RR
garfield-2003-551R
garfield-2003-589R
ottawa-2006-248B
cotton-2005-88W
stephens-2004-336R
carter-2004-452B
rogers-2010-479R
rogers-2010-56R
jackson-2005-81R
atoka-2005-40B
oklahoma-2010-7796R
muskogee-2005-581B
muskogee-2005-581B
oklahoma-2005-6907R
oklahoma-2005-6907R
creek-2008-45R
garfield-2005-124R
garfield-2005-124R
oklahoma-2012-4808R
kiowa-2012-35R
kiowa-2012-35R
comanche-2007-259R
oklahoma-2010-6536R
creek-2011-256R
creek-2011-256R
pittsburg-2007-392D
pittsburg-20006-688A
tillman-2010-92B
tillman-2010-92B
pontotoc-2012-66R
wagoner-2007-121R
logan-2009-229R
logan-2009-229R
oklahoma-2008-1953R
oklahoma-2008-1953R

comanche-2014-423R
cleveland-2015-1198 TS
cleveland-2015-1198 TS
cleveland-2014-487 TS
cleveland-2014-487 TS
washington-2014-186R
washington-2014-186R
delaware-2014-371A
garfield-2014-191R
garfield-2014-191R
garfield-2014-191R
garfield-2014-341R
garfield-2014-8R
grant-2014-23R
grant-2014-27R
haskell-2015-18R
muskogee-2014-8RR
muskogee-2016-377R
muskogee-2014-8R
seminole-2014-248S
seminole-2014-248S
lincoln-2014-180A
nowata-2014-20 ct 1
nowata-2014-20 ct 1
carter-2014-689A
carter-2017-240S
rogers-2015-41 CT1
rogers-2015-41 CT 2
rogers-2015-41 CT 3
payne-2015-108R
payne-2013-299R
caddo-2015-20 RSS
okmulgee-2012-353R
oklahoma-2014-3312R
jackson-2015-9S
wagoner-2013-440R
comanche-2015-118R
comanche-2014-512R
tulsa-2015-3393R
tulsa-2015-3975R
tulsa-2015-3975R
tulsa-2015-4098R
tulsa-2016-1R
tulsa-2016-1R
tulsa-2016-1459R
oklahoma-2015-5650R
oklahoma-2015-5650R
oklahoma-2015-4877R
oklahoma-2015-4877R
oklahoma-2015-4608R
mcintosh-2015-67R
lincoln-2015-256R
mayes-2013-267AD
mayes-2013-267AD
cle

cleveland-2015-1760S
cleveland-2016-538S
tillman-2019-18S
marshall-2015-122R
marshall-2015-122R
delaware-2018-257A
pottawatomie-2017-444S
pottawatomie-2018-126S
ottawa-2016-379S
ottawa-2016-510S
sequoyah-2017-775R
sequoyah-2017-775R
sequoyah-2015-656R
carter-2019-204B
delaware-2017-170A
delaware-2017-170A
delaware-2017-170A
delaware-2017-170A
delaware-2017-170A
delaware-2017-170A
texas-2019-F
stephens-2019-55S
stephens-2019-55S
texas-None-20190C07437
delaware-2011-311R
delaware-2016-91R
delaware-2017-99S
delaware-2018-90S
haskell-2019-24S
haskell-2019-24S
bryan-2016-206S
pittsburg-2020-65A
pontotoc-2018-311S
pontotoc-2016-477S
pontotoc-2018-352S
pontotoc-2018-438S
pontotoc-2018-438S
pontotoc-2019-72S
pontotoc-2019-42S
pontotoc-2019-42S
pontotoc-2019-42S
lincoln-2019-30A


,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS,oscn_county,oscn_year,oscn_number,oscn_valid
4527,8359,163-2-401,OKLAHOMA COUNTY COURT,10-JUN-87,87-615,5.0,NaN,MESLES,ARDELL,NaN,...,NaN,NaN,NaN,NaN,NaN,Inactive,oklahoma,1987,615,True
36683,10337,021-1720,MCCURTAIN COUNTY COURT,04-OCT-79,79-167,1.0,2.0,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,mccurtain,1979,167,False
36684,10337,121-1435,MCCURTAIN COUNTY COURT,05-OCT-79,77-134,2.0,NaN,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,mccurtain,1977,134,False
36685,10337,221-1116,POTTAWATOMIE COUNTY COURT,05-SEP-79,79-220,NaN,2.0,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1979,220,False
36686,10337,421-1431,POTTAWATOMIE COUNTY COURT,03-JAN-92,91-197,10.0,NaN,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1991,197,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334839,85741,821-1713,LINCOLN COUNTY COURT,24-FEB-20,19-30A,NaN,7.0,BARNETT,NOVA,L,...,BLACK,5.0,8.0,240.0,BROWN,Inactive,lincoln,2019,30A,None
1334840,85741,963-2-401,LINCOLN COUNTY COURT,02-MAR-20,18-156,NaN,10.0,BARNETT,NOVA,L,...,BLACK,5.0,8.0,240.0,BROWN,Inactive,lincoln,2018,156,True
1335188,99018,421-798,WASHINGTON COUNTY COURT,16-MAY-84,84-6869,20.0,NaN,HESS,BILLY,JOE,...,BROWN,5.0,8.0,145.0,BLUE,Inactive,washington,1984,6869,False
1335189,99018,421-645,WASHINGTON COUNTY COURT,16-MAY-84,84-1686,10.0,NaN,HESS,BILLY,JOE,...,BROWN,5.0,8.0,145.0,BLUE,Inactive,washington,1984,1686,False


In [39]:
filter_on_valid = combined_data.dropna(subset=['oscn_valid'])  
filter_on_valid = filter_on_valid.loc[filter_on_valid['oscn_valid'], :]
filter_on_valid

,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS,oscn_county,oscn_year,oscn_number,oscn_valid
4527,8359,163-2-401,OKLAHOMA COUNTY COURT,10-JUN-87,87-615,5.0,NaN,MESLES,ARDELL,NaN,...,NaN,NaN,NaN,NaN,NaN,Inactive,oklahoma,1987,615,True
36689,10337,421-1435,POTTAWATOMIE COUNTY COURT,14-APR-98,97-388,20.0,NaN,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1997,388,True
36690,10337,421-1720,CLEVELAND COUNTY COURT,03-JUN-91,91-546,NaN,2.00,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,cleveland,1991,546,True
36691,10337,421-1435,POTTAWATOMIE COUNTY COURT,06-MAR-08,1997-388,NaN,3.66,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,pottawatomie,1997,388,True
36693,10337,547-11-902,TULSA COUNTY COURT,31-OCT-00,2000-3557,4.0,NaN,SAWYER,FRANK,N,...,UNK,5.0,1.0,NaN,BLACK,Inactive,tulsa,2000,3557,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334765,85728,763-2-401,CANADIAN COUNTY COURT,13-JAN-20,cf-2019-458,NaN,10.00,REX,LAWRENCE,J,...,BLACK,5.0,8.0,180.0,BROWN,Inactive,canadian,2019,458,True
1334766,85728,763-2-401,SEQUOYAH COUNTY COURT,26-FEB-20,CF-19-315,NaN,10.00,REX,LAWRENCE,J,...,BLACK,5.0,8.0,180.0,BROWN,Inactive,sequoyah,2019,315,True
1334837,85741,421-175,CREEK COUNTY COURT BRISTOW,19-FEB-20,20-9,NaN,5.00,BARNETT,NOVA,L,...,BLACK,5.0,8.0,240.0,BROWN,Inactive,creek,2020,9,True
1334838,85741,547-4-103,LINCOLN COUNTY COURT,24-FEB-20,19-15,NaN,2.00,BARNETT,NOVA,L,...,BLACK,5.0,8.0,240.0,BROWN,Inactive,lincoln,2019,15,True
